# Ocean thermal forcing -- Verjans refactored with xarray
Clean ocean TF workflow for deployment on CCR.

10 Oct 2024 | EHU
- 15 Oct: try with CESM rather than IPSL, for now.  The [IPSL tripolar grid](https://cmc.ipsl.fr/international-projects/cmip5/ipsl-contribution-to-cmip5-faq/) is a complication to deal with in the next revision.
- 16 Oct: CESM2 is successful (in 2000-2014 and 1950-1999 examples)! Next try implementing `xr.mfdataset`, or else a for loop, to include all historical data together.  MFdataset may complicate the write-out on GHub but could be good for CCR.
- 24 Oct: Found that use of an unconventional fill value may have been messing with processing.  Removed fill value of 1.1e20 from the masking command `tf_out = fftf.where(gld_ds.thetao<1e10)` because xarray handles NaNs by default.

In [45]:
import os
import sys
import glob
import copy
import csv
import numpy as np
import netCDF4 as nc
import xarray as xr
import dask
from datetime import datetime

from codeFunctions import freezingPoint

In [46]:
### Settings for this run
saveBoxGreenlandNC = True
cwd                = os.getcwd()+'/'

SelModel = 'CESM2'

DirThetaoNC = f'/home/theghub/ehultee/projects/cmipdata/files/'
DirSoNC     = f'/home/theghub/ehultee/projects/cmipdata/files/'
DirSaveNC   = f'{cwd}../data/'

### Select experiment ###
To2015hist                 = True
To2100histssp585           = False
To2100histssp126           = False

if(To2015hist):
    Experiments = ['historical']
    DatesCut    = [2015]
elif(To2100histssp585): 
    Experiments = ['historical','ssp585']
    DatesCut    = [2015,2100]
elif(To2100histssp126): 
    Experiments = ['historical','ssp126']
    DatesCut    = [2015,2100]
nExp          = len(Experiments)
depthUnitConv = 1.0 #initialize depth unit converter

### Limits of Greenland domain ###
limN           = 86.0 ## degrees N latitude
limS           = 57.0 ## degrees N latitude
limE           = 4.0 ## degrees E latitude
limW           = 274.0 ## degrees E latitude
## CHECK: confirm that output shows up within this W-E box and not its E-W complement
limDp          = 1200.0
depthSubSample = 1



In [47]:
### 
nExp          = len(Experiments)
depthUnitConv = 1.0 #initialize depth unit converter

if(SelModel=='MIROCES2L'):
    dim2d              = True
    if(To2015hist):
        ls_members     = [f'r{id}' for id in range(1,30+1)]
    elif(To2100histssp585 or To2100histssp126):
        ls_members     = [f'r{id}' for id in range(1,10+1)]
    namelat            = 'latitude'
    namelon            = 'longitude'
    namez              = 'lev'
    datesendhist       = np.array(['201412'])
    if(To2100histssp585):
        datesendssp585     = np.array(['210012'])
    if(To2100histssp126):
        datesendssp126     = np.array(['210012'])
        
if(SelModel=='IPSL-CM6A-LR'):
    dim2d              = True
    if(To2015hist):
        ls_members     = [f'r{id}' for id in range(1,32+1)]
        ls_members.remove('r2') #no r2 member for IPSLCM6A
    elif(To2100histssp585 or To2100histssp126):
        ls_members     = ['r1'] #,'r3','r4','r6','r14']
    namelat            = 'nav_lat'
    namelon            = 'nav_lon'
    namez              = 'olevel'
    datesRef           = [1850.0,2015.0,2040.0] 
    datesendhist       = np.array(['194912','201412'])
    if(To2100histssp585):
        datesendssp585     = np.array(['210012'])
    if(To2100histssp126):
        datesendssp126     = np.array(['210012'])

else:
    print(f'Error script not implemented yet for {SelModel}')

# nMemb           = len(ls_members)

Error script not implemented yet for CESM2


### List the files to be read 

In [48]:
ThetaFiles_test = []
SoFiles_test = []
for expt in Experiments:
    fpath1 = DirThetaoNC+'thetao_Omon_{}_{}_'.format(SelModel, expt)
    print(fpath1)
    fpath2 = DirSoNC+'so_Omon_{}_{}_'.format(SelModel, expt)
    th_temp = glob.glob(f'{fpath1}*.nc')
    s_temp = glob.glob(f'{fpath2}*.nc')
    ThetaFiles_test += th_temp ##concat the glob lists
    SoFiles_test += s_temp

/home/theghub/ehultee/projects/cmipdata/files/thetao_Omon_CESM2_historical_


Confirm that the list is not empty.  If it is, something has gone wrong in the directory access or in the generation of names.

In [49]:
ThetaFiles_test

['/home/theghub/ehultee/projects/cmipdata/files/thetao_Omon_CESM2_historical_r11i1p1f1_gr_195001-199912.nc',
 '/home/theghub/ehultee/projects/cmipdata/files/thetao_Omon_CESM2_historical_r11i1p1f1_gr_185001-189912.nc',
 '/home/theghub/ehultee/projects/cmipdata/files/thetao_Omon_CESM2_historical_r11i1p1f1_gr_190001-194912.nc',
 '/home/theghub/ehultee/projects/cmipdata/files/thetao_Omon_CESM2_historical_r11i1p1f1_gr_200001-201412.nc']

### Load using a `with`-statement, to release memory as much as possible

Specify the paths of the `thetao` and `so` variables -- ensure they come from the same GCM (`SelModel`) and time period.  Use a `with` statement to read in, trim, and close the parent datasets.  This should leave us with the trimmed datasets `gld_ds` and `gld_so` to work with below.

In [50]:
path0 = [f for f in ThetaFiles_test if '2000' in f][0] ## thetao
path1 = [f for f in SoFiles_test if '2000' in f][0] ## salinity

## load in and trim thetao
with xr.open_dataset(path0, chunks={'lev':10}) as ds:
    ## trim to Greenland bounding box
    include_lat = (ds.lat>=limS) & (ds.lat <=limN)
    include_lon = np.logical_or(((ds.lon%360)<=limE),((ds.lon %360) >=limW)) 
    ## modulo 360 to account for lon going -180 to 180 or 0-360
    
    with dask.config.set(**{'array.slicing.split_large_chunks': True}): ## mitigate performance problem with slicing
        gld_ds = ds.where(include_lat & include_lon, drop=True)

## load and trim so
with xr.open_dataset(path1, chunks={'lev':10}) as ds1:
    ## trim to Greenland bounding box
    include_lat = (ds1.lat>=limS) & (ds1.lat <=limN)
    include_lon = np.logical_or(((ds1.lon%360)<=limE),((ds1.lon %360) >=limW))
    
    with dask.config.set(**{'array.slicing.split_large_chunks': True}): ## mitigate performance problem with slicing
        gld_so = ds1.where(include_lat & include_lon, drop=True) ## trim to Gld
    


/apps/share64/debian10/anaconda/anaconda-7/envs/geospatial-2021-09/lib/python3.7/site-packages/xarray/conventions.py:520: SerializationWarning: variable 'thetao' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/apps/share64/debian10/anaconda/anaconda-7/envs/geospatial-2021-09/lib/python3.7/site-packages/xarray/conventions.py:520: SerializationWarning: variable 'so' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,


In [51]:
gld_ds

<xarray.Dataset>
Dimensions:    (time: 180, d2: 2, lat: 29, lon: 90, lev: 33)
Coordinates:
  * time       (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lev        (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat        (lat) float64 57.5 58.5 59.5 60.5 61.5 ... 82.5 83.5 84.5 85.5
  * lon        (lon) float64 0.5 1.5 2.5 3.5 274.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: d2
Data variables:
    time_bnds  (time, d2, lat, lon) object dask.array<chunksize=(180, 2, 29, 90), meta=np.ndarray>
    lev_bnds   (lev, d2, lat, lon) float64 dask.array<chunksize=(10, 2, 29, 90), meta=np.ndarray>
    lat_bnds   (lat, d2, lon) float64 dask.array<chunksize=(29, 2, 90), meta=np.ndarray>
    lon_bnds   (lon, d2, lat) float64 dask.array<chunksize=(90, 2, 29), meta=np.ndarray>
    thetao     (time, lev, lat, lon) float32 dask.array<chunksize=(180, 10, 29, 30), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    sub_experiment_id:      none
    table_id:               Omon
    tracking_id:            hdl:21.14100/8a968f2d-d094-4d9d-b575-1838314e0e2b
    variable_id:            thetao
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1

### Compute the ocean thermal forcing

In [52]:
fp = xr.apply_ufunc(freezingPoint, gld_so.so, gld_so.lev, dask='parallelized',
                   dask_gufunc_kwargs={'allow_rechunk':True})
fftf = gld_ds.thetao - fp

In [53]:
## mask and apply a fill value
tf_out = fftf.where(gld_ds.thetao<1e10) ## apply Vincent's fill value of 1.1e20
## actually, just let xarray do its native processing with NaNs.

In [54]:
tf_out.assign_attrs(standard_name='TF',
                    long_name='Ocean thermal forcing',
                    # fillvalue=1.1e20,
                    latbounds=[limS, limN],
                    lonbounds=[limW,limE])

<xarray.DataArray (time: 180, lev: 33, lat: 29, lon: 90)>
dask.array<where, shape=(180, 33, 29, 90), dtype=float64, chunksize=(180, 10, 29, 30), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat      (lat) float64 57.5 58.5 59.5 60.5 61.5 ... 81.5 82.5 83.5 84.5 85.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 274.5 ... 356.5 357.5 358.5 359.5
Attributes:
    standard_name:  TF
    long_name:      Ocean thermal forcing
    latbounds:      [57.0, 86.0]
    lonbounds:      [274.0, 4.0]

In [55]:
now = datetime.now()
ds_temp = tf_out.to_dataset(name='TF')
# ds_temp.TF.attrs = tf_out.attrs
ds_out = ds_temp.assign_attrs(title='Ocean thermal forcing for {}'.format(SelModel),
                             summary='TF computed following Verjans code, in a bounding' + 
                              ' box around Greenland, for ISMIP7 Greenland forcing',
                             institution='NASA Goddard Space Flight Center',
                             creation_date=now.strftime('%Y-%m-%d %H:%M:%S'))

ds_out

<xarray.Dataset>
Dimensions:  (time: 180, lev: 33, lat: 29, lon: 90)
Coordinates:
  * time     (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat      (lat) float64 57.5 58.5 59.5 60.5 61.5 ... 81.5 82.5 83.5 84.5 85.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 274.5 ... 356.5 357.5 358.5 359.5
Data variables:
    TF       (time, lev, lat, lon) float64 dask.array<chunksize=(180, 10, 29, 30), meta=np.ndarray>
Attributes:
    title:          Ocean thermal forcing for CESM2
    summary:        TF computed following Verjans code, in a bounding box aro...
    institution:    NASA Goddard Space Flight Center
    creation_date:  2024-10-24 20:43:11

In [56]:
ds_out.info()

xarray.Dataset {
dimensions:
	time = 180 ;
	lev = 33 ;
	lat = 29 ;
	lon = 90 ;

variables:
	object time(time) ;
		time:axis = T ;
		time:bounds = time_bnds ;
		time:standard_name = time ;
		time:title = time ;
		time:type = double ;
	float64 lev(lev) ;
		lev:axis = Z ;
		lev:bounds = lev_bnds ;
		lev:long_name = ocean model level ;
		lev:positive = down ;
		lev:standard_name = olevel ;
		lev:units = m ;
	float64 lat(lat) ;
		lat:axis = Y ;
		lat:bounds = lat_bnds ;
		lat:long_name = latitude ;
		lat:standard_name = latitude ;
		lat:units = degrees_north ;
	float64 lon(lon) ;
		lon:axis = X ;
		lon:bounds = lon_bnds ;
		lon:long_name = longitude ;
		lon:standard_name = longitude ;
		lon:units = degrees_east ;
	float64 TF(time, lev, lat, lon) ;

// global attributes:
	:title = Ocean thermal forcing for CESM2 ;
	:summary = TF computed following Verjans code, in a bounding box around Greenland, for ISMIP7 Greenland forcing ;
	:institution = NASA Goddard Space Flight Center ;
	:creation_dat

### Write NetCDF out
Attempt a write-out of this file.  The regular grid version for matplotlib is estimated at 400 MB - should be possible.

Note we can't use Vincent's `DatesCut` here, so try using the year tag from the input files instead.

In [57]:
out_path = '/home/theghub/ehultee/data/'
year_tag = path0.strip('.nc').split('_')[-1] ## take the year tag from the GCM input (only one of the two input DS, but we have tried to make them match!)
out_fn = out_path + 'tf-{}-{}-v4_no_intermed_compute.nc'.format(SelModel, year_tag)

from dask.diagnostics import ProgressBar

with ProgressBar():
    ds_out.to_netcdf(path=out_fn)

[########################################] | 100% Completed | 10.6s


### Remove later: check the output

In [ ]:
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list

In [58]:
out_path = '/home/theghub/ehultee/data/'
f_in = out_path + 'tf-{}-200001-201412-v4_no_intermed_compute.nc'.format(SelModel)

ds_new = xr.open_dataset(f_in)

In [59]:
ds_new

<xarray.Dataset>
Dimensions:  (time: 180, lev: 33, lat: 29, lon: 90)
Coordinates:
  * time     (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat      (lat) float64 57.5 58.5 59.5 60.5 61.5 ... 81.5 82.5 83.5 84.5 85.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 274.5 ... 356.5 357.5 358.5 359.5
Data variables:
    TF       (time, lev, lat, lon) float64 ...
Attributes:
    title:          Ocean thermal forcing for CESM2
    summary:        TF computed following Verjans code, in a bounding box aro...
    institution:    NASA Goddard Space Flight Center
    creation_date:  2024-10-24 20:43:11

In [60]:
tf_tavg = ds_new.TF.mean(dim='time') 
## TODO: remember to re-run the above and name the output variable!
tf_tavg

<xarray.DataArray 'TF' (lev: 33, lat: 29, lon: 90)>
array([[[13.3183118 , 13.24183313, 13.15024916, ...,         nan,
         13.39216012, 13.36355305],
        [13.28129936, 13.18653209, 13.06264397, ..., 13.51482948,
         13.44842328, 13.36339421],
        [13.16549972, 13.0565375 , 12.93610441, ..., 13.62933732,
         13.41291354, 13.27217186],
        ...,
        [ 0.07004895,  0.07077562,  0.07119395, ...,  0.06802135,
          0.0683985 ,  0.06914584],
        [ 0.05245069,  0.05246685,  0.05301159, ...,  0.05277584,
          0.052718  ,  0.05259279],
        [ 0.03909557,  0.03888317,  0.03862714, ...,  0.03945   ,
          0.03935935,  0.03923941]],

       [[13.29972081, 13.22381365, 13.12975535, ...,         nan,
         13.36756905, 13.3421577 ],
        [13.26431459, 13.16941283, 13.04386782, ..., 13.49188605,
         13.42895732, 13.34571259],
        [13.14844157, 13.0390151 , 12.91612547, ..., 13.61045037,
         13.39500304, 13.25499952],
...
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]])
Coordinates:
  * lev      (lev) float64 0.0 10.0 20.0 30.0 ... 4e+03 4.5e+03 5e+03 5.5e+03
  * lat      (lat) float64 57.5 58.5 59.5 60.5 61.5 ... 81.5 82.5 83.5 84.5 85.5
  * lon      (lon) float64 0.5 1.5 2.5 3.5 274.5 ... 356.5 357.5 358.5 359.5

In [61]:
tf_tavg.sel(lev=0.0).mean(skipna=True)

<xarray.DataArray 'TF' ()>
array(4.53876719)
Coordinates:
    lev      float64 0.0

In [ ]:
import matplotlib.pyplot as plt
ax = plt.axes(projection=ccrs.Robinson())
tf_tavg.sel(lev=0.0).plot(ax=ax, transform=ccrs.PlateCarree(), x='lon', y='lat') ## specify x and y coordinates
ax.coastlines(); ax.gridlines();

In [ ]:
tf_tavg.sel(lev=0.0).plot()